In [1]:
import pandas as pd 
import numpy as np
import datetime
import Analysis
import pandas_ta as ta
from nselib import capital_market
from nselib import derivatives
import yfinance as yf

In [ ]:
# df = pd.read_csv('final_df.csv')

In [ ]:
url ="https://raw.githubusercontent.com/AshoksInsights/Stock_analysis/refs/heads/main/final_df.csv"

In [10]:
df = pd.read_csv(url)
df['Fetch_Time'] = pd.to_datetime(df['Fetch_Time'])

In [11]:
current_price = yf.download('^NSEI',period='1d',multi_level_index=False,ignore_tz=True).reset_index()['Close'].values[0]

[*********************100%***********************]  1 of 1 completed


In [27]:
df = df[(df['Strike_Price'] >= current_price -500) & (df['Strike_Price'] <= current_price +500)]

In [28]:
combination = df[['Strike_Price','Expiry_Date']].drop_duplicates()

In [ ]:
# new = df[(df['Strike_Price'] == 24550) & (df['Expiry_Date']=='2025-05-08')]
# new['Date'] = new['Fetch_Time'].dt.date
# chk = new[new['Date'] == datetime.date.today()][['Fetch_Time','CALLS_LTP']]
# new[new['Date'] == datetime.date.today()]
# chk1 = chk.resample('5min',on='Fetch_Time').agg({'CALLS_LTP':['first', 'min', 'max', 'last']}).reset_index()
# chk1.columns = ['datetime', 'Open', 'Low', 'High', 'Close']
# chk1['smma'] = Analysis.calculate_smma(chk1, period=7)
# chk1['rsi'] = ta.rsi(chk1['Close'])
# chk1['dema'] = ta.dema(chk1['Close'], length=10)
# chk1['super_trend'] = ta.supertrend(chk1['High'], chk1['Low'], chk1['Close'], length=10, multiplier=3)['SUPERT_10_3.0']
# chk1['super_trend_color'] = np.where(chk1['Close'] > chk1['super_trend'] , "green" , "red")
# chk1['BBM'] = ta.bbands(close = chk1['Close'], length=20, std=2)['BBM_20_2.0']
# chk1['changeover'] = np.insert(np.diff(np.where(chk1['super_trend_color'] == 'green' , 1 , 0)),0,0)
# chk1['condition'] = np.insert(np.diff(np.where((chk1['BBM'] < chk1['smma']) & (chk1['smma'] < chk1['dema']) ,1,0)),0,0)
# chk1 = chk1.iloc[np.where(((chk1['changeover'] == 1) & (chk1['condition'] == 1)))] 

/tmp/ipykernel_1569/788553029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['Date'] = new['Fetch_Time'].dt.date


In [42]:
dff = []
for i, row in combination.iterrows():
   try:
      strike = row['Strike_Price']
      expiry = row['Expiry_Date']
      new = df[(df['Strike_Price'] == strike) & (df['Expiry_Date']== expiry)].copy()
      new['Date'] = new['Fetch_Time'].dt.date
      # chk = new[new['Date'] == datetime.date.today()][['Fetch_Time','CALLS_LTP']]
      chk = new[['Fetch_Time','CALLS_LTP']]
      new[new['Date'] == datetime.date.today()]
      chk1 = chk.resample('5min',on='Fetch_Time').agg({'CALLS_LTP':['first', 'min', 'max', 'last']}).reset_index()
      chk1.columns = ['datetime', 'Open', 'Low', 'High', 'Close']
      chk1['smma'] = Analysis.calculate_smma(chk1, period=7)
      chk1['rsi'] = ta.rsi(chk1['Close'])
      chk1['dema'] = ta.dema(chk1['Close'], length=10)
      chk1['super_trend'] = ta.supertrend(chk1['High'], chk1['Low'], chk1['Close'], length=10, multiplier=3)['SUPERT_10_3.0']
      chk1['super_trend_color'] = np.where(chk1['Close'] > chk1['super_trend'] , "green" , "red")
      chk1['BBM'] = ta.bbands(close = chk1['Close'], length=20, std=2)['BBM_20_2.0']
      chk1['changeover'] = np.insert(np.diff(np.where(chk1['super_trend_color'] == 'green' , 1 , 0)),0,0)
      chk1['condition'] = np.insert(np.diff(np.where((chk1['BBM'] < chk1['smma']) & (chk1['smma'] < chk1['dema']) ,1,0)),0,0)
      chk1 = chk1.iloc[np.where(((chk1['changeover'] == 1) & (chk1['condition'] == 1)))] 
      chk1['strike'] = strike
      chk1['expiry'] = expiry
      chk1 = chk1[chk1['Date'] == datetime.date.today()]
      chk1 = chk1[['datetime', 'Close','strike', 'expiry']]
      chk1['type'] = 'CE'
      dff.append(chk1)
   except:
      chk1 = pd.DataFrame(columns=['datetime', 'Close','strike', 'expiry','type'])

In [14]:
# df[df['Fetch_Time'].dt.date.isin(df['Fetch_Time'].dt.date.unique()[-2:])]

In [43]:
try:
    calls = pd.concat(dff)
except:
    calls = pd.DataFrame(columns=['datetime', 'Close','strike', 'expiry','type'])

In [44]:
calls

,datetime,Close,strike,expiry,type


In [7]:
pd.read_csv(url)

,Fetch_Time,Symbol,Expiry_Date,CALLS_LTP,Strike_Price,PUTS_LTP
0,30-Apr-2025 09:26:45,NIFTY,2025-12-24,0.00,13000,0.0
1,30-Apr-2025 09:26:45,NIFTY,2025-06-26,0.00,14000,0.0
2,30-Apr-2025 09:26:45,NIFTY,2026-12-31,0.00,14000,0.0
3,30-Apr-2025 09:26:45,NIFTY,2025-06-26,0.00,15000,0.0
4,30-Apr-2025 09:26:45,NIFTY,2025-12-24,0.00,15000,0.0
...,...,...,...,...,...,...
545754,02-May-2025 14:27:24,NIFTY,2029-06-28,0.00,29000,0.0
545755,02-May-2025 14:27:24,NIFTY,2029-12-27,0.00,29000,0.0
545756,02-May-2025 14:27:24,NIFTY,2025-06-26,10.15,30000,0.0
545757,02-May-2025 14:27:24,NIFTY,2025-12-24,67.95,30000,0.0


In [85]:
df[df['Strike_Price'] >= current_price]

,Fetch_Time,Symbol,Expiry_Date,CALLS_LTP,Strike_Price,PUTS_LTP
579,2025-04-30 09:26:45,NIFTY,2025-04-30,14.20,24350,76.00
580,2025-04-30 09:26:45,NIFTY,2025-05-08,208.55,24350,250.55
581,2025-04-30 09:26:45,NIFTY,2025-05-15,318.60,24350,339.90
582,2025-04-30 09:26:45,NIFTY,2025-05-22,0.00,24350,370.00
583,2025-04-30 09:26:45,NIFTY,2025-05-29,463.95,24350,424.55
...,...,...,...,...,...,...
545754,2025-05-02 14:27:24,NIFTY,2029-06-28,0.00,29000,0.00
545755,2025-05-02 14:27:24,NIFTY,2029-12-27,0.00,29000,0.00
545756,2025-05-02 14:27:24,NIFTY,2025-06-26,10.15,30000,0.00
545757,2025-05-02 14:27:24,NIFTY,2025-12-24,67.95,30000,0.00
